# Homework 1: Linear Regression

本次目標：由前 9 個小時的 18 個 features (包含 PM2.5)預測的 10 個小時的 PM2.5。
若有任何問題，歡迎來信至助教信箱 ntu-ml-2020spring-ta@googlegroups.com

# **Load 'train.csv'**
train.csv 的資料為 12 個月中，每個月取 20 天，每天 24 小時的資料(每小時資料有 18 個 features)。

In [2]:
import sys  # Import the sys module for system-specific parameters and functions
import pandas as pd  # Import pandas for data manipulation
import numpy as np  # Import numpy for numerical operations
# from google.colab import drive  # Import drive module from google.colab for Google Drive operations

# Download the dataset from Google Drive using its unique ID and save it as 'data.zip'
# !gdown --id '1wNKAxQ29G15kgpBy_asjTcZRRgmsCZRm' --output data.zip

# Unzip the downloaded 'data.zip' file to access the dataset
# !unzip data.zip

# 因為我不是在 google colab 上執行，所以不需要使用 google drive
# 我直接下載 train.csv 到本地端，所以不需要使用 gdown 這個套件

# Read the CSV file 'train.csv' using pandas, assuming the file is encoded in 'big5' encoding
# data = pd.read_csv('gdrive/My Drive/hw1-regression/train.csv', header = None, encoding = 'big5')
data = pd.read_csv('./data/train.csv', encoding = 'big5')


把 data 印出來看看

In [3]:
print(type(data))
print(data)
print()
print("===========")
print()
print(data.iloc[0])

<class 'pandas.core.frame.DataFrame'>
              日期  測站          測項     0     1     2     3     4     5     6  \
0       2014/1/1  豐原    AMB_TEMP    14    14    14    13    12    12    12   
1       2014/1/1  豐原         CH4   1.8   1.8   1.8   1.8   1.8   1.8   1.8   
2       2014/1/1  豐原          CO  0.51  0.41  0.39  0.37  0.35   0.3  0.37   
3       2014/1/1  豐原        NMHC   0.2  0.15  0.13  0.12  0.11  0.06   0.1   
4       2014/1/1  豐原          NO   0.9   0.6   0.5   1.7   1.8   1.5   1.9   
...          ...  ..         ...   ...   ...   ...   ...   ...   ...   ...   
4315  2014/12/20  豐原         THC   1.8   1.8   1.8   1.8   1.8   1.7   1.7   
4316  2014/12/20  豐原       WD_HR    46    13    61    44    55    68    66   
4317  2014/12/20  豐原  WIND_DIREC    36    55    72   327    74    52    59   
4318  2014/12/20  豐原  WIND_SPEED   1.9   2.4   1.9   2.8   2.3   1.9   2.1   
4319  2014/12/20  豐原       WS_HR   0.7   0.8   1.8     1   1.9   1.7   2.1   

      ...    14    15    

# **Preprocessing** 
取需要的數值部分，將 'RAINFALL' 欄位全部補 0。
另外，如果要在 colab 重覆這段程式碼的執行，請從頭開始執行(把上面的都重新跑一次)，以避免跑出不是自己要的結果（若自己寫程式不會遇到，但 colab 重複跑這段會一直往下取資料。意即第一次取原本資料的第三欄之後的資料，第二次取第一次取的資料掉三欄之後的資料，...）。

In [4]:
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()

印出 `raw_data` 看看

In [5]:
print(raw_data)
print(type(raw_data))
print(raw_data[0])
print(type(raw_data[0]))
print(raw_data[0][2])
print(type(raw_data[0][2]))
# raw_data 是一個 2D numpy.ndarray
# 其中每一個元素都是一個 string

[['14' '14' '14' ... '15' '15' '15']
 ['1.8' '1.8' '1.8' ... '1.8' '1.8' '1.8']
 ['0.51' '0.41' '0.39' ... '0.35' '0.36' '0.32']
 ...
 ['36' '55' '72' ... '118' '100' '105']
 ['1.9' '2.4' '1.9' ... '1.5' '2' '2']
 ['0.7' '0.8' '1.8' ... '1.6' '1.8' '2']]
<class 'numpy.ndarray'>
['14' '14' '14' '13' '12' '12' '12' '12' '15' '17' '20' '22' '22' '22'
 '22' '22' '21' '19' '17' '16' '15' '15' '15' '15']
<class 'numpy.ndarray'>
14
<class 'str'>


# **Extract Features (1)**
![圖片說明](./explain_imgs/download.png)
![圖片說明](./explain_imgs/ex.png)
![自己畫的圖片說明](./explain_imgs/self-ex.jpeg)

將原始 4320 * 18 的資料依照每個月分重組成 12 個 18 (features) * 480 (hours) 的資料。 

In [6]:
month_data = {}
for month in range(12): # 12 months
    sample = np.empty([18, 480]) # 18 features, 20 days * 24 hoursm, np.empty() default dtype is float64
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :] # implicit type conversion
    month_data[month] = sample

把 `month_data` 印出來看看

In [7]:
print(month_data)
print(type(month_data))
print(type(month_data[0]))
print(month_data[0].shape)
print(type(month_data[0][0]))
print(month_data[0][0].shape)
print(type(month_data[0][0][0]))

{0: array([[14.  , 14.  , 14.  , ..., 14.  , 13.  , 13.  ],
       [ 1.8 ,  1.8 ,  1.8 , ...,  1.8 ,  1.8 ,  1.8 ],
       [ 0.51,  0.41,  0.39, ...,  0.34,  0.41,  0.43],
       ...,
       [35.  , 79.  ,  2.4 , ..., 48.  , 63.  , 53.  ],
       [ 1.4 ,  1.8 ,  1.  , ...,  1.1 ,  1.9 ,  1.9 ],
       [ 0.5 ,  0.9 ,  0.6 , ...,  1.2 ,  1.2 ,  1.3 ]]), 1: array([[ 15.  ,  14.  ,  14.  , ...,   8.4 ,   8.  ,   7.6 ],
       [  1.8 ,   1.8 ,   1.7 , ...,   1.7 ,   1.7 ,   1.7 ],
       [  0.27,   0.26,   0.25, ...,   0.36,   0.35,   0.32],
       ...,
       [113.  , 109.  , 104.  , ...,  72.  ,  65.  ,  69.  ],
       [  2.3 ,   2.2 ,   2.6 , ...,   1.9 ,   2.9 ,   1.5 ],
       [  2.5 ,   2.2 ,   2.2 , ...,   0.9 ,   1.6 ,   1.1 ]]), 2: array([[ 18.  ,  18.  ,  18.  , ...,  14.  ,  13.  ,  13.  ],
       [  1.8 ,   1.8 ,   1.8 , ...,   1.8 ,   1.8 ,   1.8 ],
       [  0.39,   0.36,   0.4 , ...,   0.42,   0.47,   0.49],
       ...,
       [103.  , 128.  , 115.  , ...,  60.  ,  94.  ,  53

# **Extract Features (2)**
![alt text](./explain_imgs/ex2.png)
![alt text](./explain_imgs/download-2.png)
![](./explain_imgs/extract2.jpg)

每個月會有 480hrs，每 9 小時形成一個 data，每個月會有 471 個 data，故總資料數為 471 * 12 筆，而每筆 data 有 9 * 18 的 features (一小時 18 個 features * 9 小時)。

對應的 target 則有 471 * 12 個(第 10 個小時的 PM2.5)

一個月有 471 筆資料怎麼算的

先算一個月有幾個小時：
$24 \frac{個小時}{天}\times20 \frac{天}{月} = 480 個小時$

***最後一個小時為 20 號的 23.***

10 個小時為一筆資料，包到最後一包為 20 號 14.~23.

即 第 (20-1)*24 + 14 = 470 hr 到 第 (20-1)*24 + 23 = 479 hr

In [8]:
x = np.empty([12 * 471, 18 * 9], dtype = float)
y = np.empty([12 * 471, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14: # 第 20 天的最後第 14 小時是最後一筆資料，因為 10 小時為一組資料， 14 + 9 = 23 剛好在 0. ~ 23. 範圍內
                continue
            # 每個月有 471 組資料，所以 day 跟 hour 跑完一輪就是 471 組資料
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
print(x)
print(y)

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


# **Standardize (1)**

[standardization vs nomalization](https://www.youtube.com/watch?v=bqhQ2LWBheQ)

這是一個議題喔 關鍵字就是 standardization vs nomalization

Gradient Calculation 要 standardize/nomalize 避免某項的影響蓋過另一項

In [9]:
mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
x

array([[-1.35825331, -1.35883937, -1.359222  , ...,  0.26650729,
         0.2656797 , -1.14082131],
       [-1.35825331, -1.35883937, -1.51819928, ...,  0.26650729,
        -1.13963133, -1.32832904],
       [-1.35825331, -1.51789368, -1.67717656, ..., -1.13923451,
        -1.32700613, -0.85955971],
       ...,
       [-0.88092053, -0.72262212, -0.56433559, ..., -0.57693779,
        -0.29644471, -0.39079039],
       [-0.7218096 , -0.56356781, -0.72331287, ..., -0.29578943,
        -0.39013211, -0.1095288 ],
       [-0.56269867, -0.72262212, -0.88229015, ..., -0.38950555,
        -0.10906991,  0.07797893]])

我的疑問：

So we don't standardize a feature if its deviation is 0??
I know that it's because we can't divided by 0
but we might then have some feature that is not standardized, wouldn't it affect the result?

ChatGPT4 回答：

You're correct in noting that the code skips standardization for features with a standard deviation of 0 to avoid division by zero. A standard deviation of 0 means that all values in the feature are identical. When this occurs, subtracting the mean (which is equal to any of the values since they're all the same) results in a feature where all values are 0. Standardizing such a feature (i.e., scaling it to have a mean of 0 and a standard deviation of 1) is not meaningful because the feature doesn't vary—it provides no information that could help differentiate between observations.

Having some features not standardized while others are can indeed affect the results of certain machine learning algorithms, especially those sensitive to the scale of the data, such as gradient descent-based methods, k-nearest neighbors, and support vector machines. These algorithms might not perform optimally if the features are on vastly different scales.

In practice, if a feature has a standard deviation of 0 (and thus no variance), it's often considered a constant feature and might be removed from the dataset before training a model because:

1. **It doesn't contribute to the model's predictive power**: Since all values are the same, the feature doesn't help the model differentiate between different observations.
2. **It can cause computational issues**: As you mentioned, dividing by zero is undefined, leading to potential errors or infinite values during computation.

However, before removing such features, it's essential to understand the data and context to ensure that the feature is indeed not informative and not just an artifact of the dataset at hand. In some cases, a feature might appear constant due to issues with data collection or a limited dataset scope but could vary in a broader or different dataset.

看來這邊只是選一個簡單作法，並沒有 Optimized。以作業的角度來想非常合理

# **Split Training Data Into "train_set" and "validation_set"**
這部分是針對作業中 report 的第二題、第三題做的簡單示範，以生成比較中用來訓練的 train_set 和不會被放入訓練、只是用來驗證的 validation_set。

In [10]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [ 0.86929969  0.70886668  0.38952809 ...  1.39110073  0.2656797
  -0.39079039]
 [ 0.71018876  0.39075806  0.07157353 ...  0.26650729 -0.39013211
  -0.39079039]
 [ 0.3919669   0.07264944  0.07157353 ... -0.38950555 -0.39013211
  -0.85955971]]
[[30.]
 [41.]
 [44.]
 ...
 [ 7.]
 [ 5.]
 [14.]]
[[ 0.07374504  0.07264944  0.07157353 ... -0.38950555 -0.85856912
  -0.57829812]
 [ 0.07374504  0.07264944  0.23055081 ... -0.85808615 -0.57750692
   0.54674825]
 [ 0.07374504  0.23170375  0.23055081 ... -0.57693779  0.54674191
  -0.1095288 ]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.3895055

# **Training**
![alt text](./explain_imgs/reg.png)
![alt text](./explain_imgs/ada.png)
![alt text](./explain_imgs/adapseu.png)

(和上圖不同處: 下面的 code 採用 Root Mean Square Error)

因為常數項的存在，所以 dimension (dim) 需要多加一欄；eps 項是避免 adagrad 的分母為 0 而加的極小數值。

每一個 dimension (dim) 會對應到各自的 gradient, weight (w)，透過一次次的 iteration (iter_time) 學習。